## 한솥 도시락 메뉴이름, 가격 크롤링하기

In [1]:
from selenium import webdriver as wb               # 브라우저 할당
from selenium.webdriver.common.by import By        # 구분자
from selenium.webdriver.common.keys import Keys    # 키보드 할당
from tqdm import tqdm
import pandas as pd
import time       # 컴퓨터 시간을 제어하는 모듈

### 1. 크롬 브라우저 켜기, 창 최대화

In [2]:
driver = wb.Chrome()

In [3]:
# 한솥도시락 홈페이지 이동
driver.get('https://www.hsd.co.kr/menu/menu_list')

- 한솥도시락 페이지를 축소하면 메뉴 구조가 바뀜
- 즉 브라우저 창의 크기가 바뀌면서 내부 요소들의 구조나 경로가 바뀔 수 도 있다는 의미
- 이를 반응형 웹이라고 하며 모바일과 웹 모두에 활용할 수 있는 웹사이트 구조
- 우리는 브라우저 창을 최대화시켜 구조 변경의 리스크를 최소화 하여 크롤링을 진행하자!

In [4]:
# 브라우저 화면 최대화
driver.maximize_window()

- <내가 한거>

In [5]:
# 9개 메뉴의 명칭과 가격을 크롤링 해보세요~!            # 내가 한거
menu = driver.find_elements(By.CLASS_NAME, 'h.fz_03')
menu_list = []
for i in menu :
    menu = i.text.strip()
    menu_list.append(menu)

menu_list

['돼지불백 정식',
 '제육볶음&돼지불백 정식',
 '돼지불백 묵은지 김치찌개 정식',
 '돼지불백 덮밥',
 '빅 돼지불백 덮밥',
 '메가 돼지불백 덮밥',
 '김치볶음밥&돼지불백',
 '왕 제육 만두(2개)',
 '반찬 돼지불백']

In [6]:
menu_price = driver.find_elements(By.CSS_SELECTOR, 'strong')              # 내가 한 거 , 왜 안나와?
menu_price_list = [] 
for i in menu_price :
    menu_price = i.text.strip()
    menu_price_list.append(menu_price)

menu_price_list

['8,500',
 '11,500',
 '12,900',
 '5,900',
 '6,500',
 '7,800',
 '7,900',
 '2,500',
 '4,500',
 '개인정보처리방침']

- <선생님이랑>
- 한솥 메뉴의 경우 메뉴명이나 가격에 대한 텍스트를 검사 버튼으로 직접 선택할 수 없음
- 이런 경우 메뉴 전체를 감싸는 태그를 검사하고 아래로 내려가면서 자식 태그나 선택자를 일일이 찾아야함

In [7]:
titles = driver.find_elements(By.CSS_SELECTOR, 'h4.h.fz_03')
titles[0].text

'돼지불백 정식'

In [8]:
titles[1].text

'제육볶음&돼지불백 정식'

In [9]:
titles[-1].text

'반찬 돼지불백'

In [10]:
len(titles)

9

In [11]:
for i in titles :
    print(i.text)

돼지불백 정식
제육볶음&돼지불백 정식
돼지불백 묵은지 김치찌개 정식
돼지불백 덮밥
빅 돼지불백 덮밥
메가 돼지불백 덮밥
김치볶음밥&돼지불백
왕 제육 만두(2개)
반찬 돼지불백


In [12]:
# 메뉴 가격
prices = driver.find_elements(By.CSS_SELECTOR, 'strong')
prices[0].text

'8,500'

In [13]:
for i in prices :
    print(i.text)

8,500
11,500
12,900
5,900
6,500
7,800
7,900
2,500
4,500
개인정보처리방침


In [14]:
len(titles), len(prices)

(9, 10)

- strong 태그로만 찾았더니 가격 맨 밑에 '개인정보처리방침' 텍스트까지 들어가 있음
- 우리가 원하는 금액에 대한 HTML요소에서 우클릭하여 Copy selector를 활용하면 해당 컨텐츠가 들어 있는 태그 및 선택자를 바로 복사할 수 있음
- Copy selector는 하나의 요소에 대한 부분이기 때문에 여러개를 추출하고 싶으면 부모 태그나 선택자부터 차례로 지워나가면서 출력되는 값을 체크하면 됨.

In [15]:
#menuList_629 > div > div.item-text > div

In [16]:
prices = driver.find_elements(By.CSS_SELECTOR, '#menuList_629 > div > div.item-text > div > strong')
for i in prices :
    print(i.text)

11,500


In [17]:
prices = driver.find_elements(By.CSS_SELECTOR, 'div > div.item-text > div > strong')    # 아이디 값 없애면 다 나옴.개인정보 처리 방침도 없어짐.
for i in prices :
    print(i.text)

8,500
11,500
12,900
5,900
6,500
7,800
7,900
2,500
4,500


In [18]:
# 기존 하던대로 태그 및 선택자를 맨 아래에서부터 보고 작성해 나가도 무방함
prices = driver.find_elements(By.CSS_SELECTOR, 'div.item-price > strong')
for i in prices :
    print(i.text)

8,500
11,500
12,900
5,900
6,500
7,800
7,900
2,500
4,500


### 금액 앞에 '가격:' 이란 텍스트 추출이 가능할까?

In [19]:
temp = driver.find_elements(By.CSS_SELECTOR, 'span.blind')
temp[0].text

''

- blind는 웹 구조에는 있지만 출력 결과에 영향을 주지 않도록 텍스트를 숨기기 위해 실무에서 종종사용하므로 크롤링이 불가

### 2. 웹 페이지 하단의 '더보기'버튼 눌러 메뉴 확장

In [20]:
# 웹 페이지 하단의 '더보기' 버튼을 누르는 코드를 작성해주세요(1회)
btn = driver.find_element(By.CSS_SELECTOR,'a.c_05' )
btn.click()

In [21]:
# 더보기 버튼이 몇 번이나 있을지 모르기 때문에 while문을 통해서 계속 눌러주는 코드
while True :
    btn = driver.find_element(By.CSS_SELECTOR,'a.c_05' )
    btn.click()
    time.sleep(2)   # 2초간 대기했다가 다음 코드 실행
    '''더보기 버튼을 누르고 페이지가 뜨는데 시간이 걸릴 수 있는데 time.sleep이 없다면 페이지가 뜨기도전에 무한정 버튼을 클릭
    하게 되어 과부하가 걸리거나 정보가 제대로 수집되지 않는 현상이 발생할 수 있음.
    더 심한 경우 서버 자체가 보안 공격으로 판단하고 접속한 클라이언트 PC의 IP를 차단해버릴 수도 있어서 항상 조심해야함.'''

# 더보기 버튼이 누를게 없어질 때까지 2초간 누르다가 더보기가 없어지고 에러가 뜸.

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=138.0.7204.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
	GetHandleVerifier [0x0x7ff61494e925+77845]
	GetHandleVerifier [0x0x7ff61494e980+77936]
	(No symbol) [0x0x7ff614709b0c]
	(No symbol) [0x0x7ff614761ba4]
	(No symbol) [0x0x7ff6147536c8]
	(No symbol) [0x0x7ff6147888ca]
	(No symbol) [0x0x7ff614752f76]
	(No symbol) [0x0x7ff614788ae0]
	(No symbol) [0x0x7ff6147b0b07]
	(No symbol) [0x0x7ff6147886a3]
	(No symbol) [0x0x7ff614751791]
	(No symbol) [0x0x7ff614752523]
	GetHandleVerifier [0x0x7ff614c2683d+3059501]
	GetHandleVerifier [0x0x7ff614c20bfd+3035885]
	GetHandleVerifier [0x0x7ff614c403f0+3164896]
	GetHandleVerifier [0x0x7ff614968c2e+185118]
	GetHandleVerifier [0x0x7ff61497053f+216111]
	GetHandleVerifier [0x0x7ff6149572d4+113092]
	GetHandleVerifier [0x0x7ff614957489+113529]
	Ge

- 코드는 맞게 짰지만 더 이상 '더보기' 버튼을 찾을 수 없는 경우에는 코드가 에러가 발생하게 됨
- '더보기' 버튼이 언제 없어질 지 모르는 상황에서 버튼이 없더라도 에러가 나지 않고 동작하도록 코드를 작성해야 함

### 3. 파이썬 예외 처리 문법 사용
- try-except 문
  - try문 : 일반적으로 동작하는 코드
  - except문 : try문이 동작하지 않는 예외의 경우 작동되는 코드

In [23]:
# 일반적으로 동작하는 코드
try : 
    while True :
        btn = driver.find_element(By.CSS_SELECTOR,'a.c_05' )
        btn.click()
        time.sleep(2)
# try문이 동작하지 않는 예외의 경우
except :
    print('더 이상 더보기 버튼이 없네요 ㅠㅠ 그래도 코드는 에러없이 이어서 동작합니다~!!')

# 더보기 버튼을 다 누르고 난 뒤(우리가 원하는 정보에 대한 HTML를 다 응답 받은 뒤) 이 때 메뉴와 가격을 추출
titles = driver.find_elements(By.CSS_SELECTOR, 'h4.h.fz_03')
prices = driver.find_elements(By.CSS_SELECTOR, 'div > div.item-text > div > strong')

titles_list = []
prices_list = []

for i in tqdm(range(len(titles))) :             # tqdm은 징행률을 표시해주는 바
    titles_list.append(titles[i].text)
    prices_list.append(prices[i].text+"원")

print("크롤링 완료!")
driver.quit()

더 이상 더보기 버튼이 없네요 ㅠㅠ 그래도 코드는 에러없이 이어서 동작합니다~!!


100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [00:02<00:00, 44.85it/s]


크롤링 완료!


In [24]:
print("메뉴 개수 : ", len(titles_list))
print("가격 개수 : ", len(prices_list))

메뉴 개수 :  106
가격 개수 :  106


### 4. 만든 메뉴와 가격에 대한 리스트의 각각의 값들을 1:1로 대응시켜 딕셔너리 형태로 만들기

In [ ]:
# zip : 길이가 동일한 두 개의 순서가 있는 집합을 1:1로 대응시켜 튜플로 묶어주는 함수
 # zip함수는 묶여진 데이터가 바로 표시되지 않아서 변수로 출력시켜도 주소값만 나올 뿐 바로 값 확인은 불가함
temp = zip(titles_list, prices_list)
temp

In [ ]:
# 메뉴와 가격을 1:1로 대응시킨 zip 형태를 딕셔너리 형 변환
titles_prices_dict = dict(zip(titles_list, prices_list))
titles_prices_dict

### 5. 모든 코드를 한 셀에 담아서 한 번에 실행하기
- 웹 크롤러나 기타 코드로 실행 파일을 만든다면 코드가 한 셀에서 시작부터 끝까지 모두 들어가 있어야 함(모듈을 만들때도 그랬듯이!)

In [ ]:
driver = wb.Chrome()
driver.get('https://www.hsd.co.kr/menu/menu_list')
driver.maximize_window()
time.sleep(2)  # 웹 페이지가 다 뜰때까지 약간의 시간을 주기

try : 
    while True :
        btn = driver.find_element(By.CSS_SELECTOR,'a.c_05' )
        btn.click()
        time.sleep(2)

except :
    print('더 이상 더보기 버튼이 없네요 ㅠㅠ 그래도 코드는 에러없이 이어서 동작합니다~!!')


titles = driver.find_elements(By.CSS_SELECTOR, 'h4.h.fz_03')
prices = driver.find_elements(By.CSS_SELECTOR, 'div > div.item-text > div > strong')

titles_list = []
prices_list = []

for i in tqdm(range(len(titles))) :           
    titles_list.append(titles[i].text)
    prices_list.append(prices[i].text+"원")

print("크롤링 완료!")
driver.quit()

print(f"메뉴 개수 : {len(titles_list)}, 가격 개수 : {len(prices_list)}")

titles_prices_dict = dict(zip(titles_list, prices_list))
titles_prices_dict